In [1]:
!pip install datasets ipywidgets
!pip install git+https://github.com/huggingface/transformers.git
!pip install accelerate

import ipywidgets as widgets
import pandas as pd
import json
import torch

from datasets import load_dataset

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-n3ztcvv_
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-n3ztcvv_
  Resolved https://github.com/huggingface/transformers.git to commit 8c12690cecbb97e187861e386f7a0ac790e4236c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 2.2 MB/s eta 0:00:002.2 MB/s eta 0:00:01
  Created wheel for transformers: filename=transformers-4.41.0.dev0-py3-none-any.whl size=9012708 sha256=91b8fb9c26e0a83dab6e6d92544e0b813984f2ee405bd55248f3a40d2e2522f2
  Stored in directory: /tmp/pip-ephem-wheel-cache-v3xd9g5m/wheels/54/cb/3f/83103de5575c534436d6a4686686dead458238dfaf1147e98d
Successfully built transformers
  Attempting uninstall: tokenizers
    Found existing installation: 

In [ ]:
# Upload the mergedJson.jsonl zip file, to preserve contents
!unzip /content/mergedJson.jsonl.zip
!sha1sum /content/mergedJson.jsonl
# Should be 7b0fa963ec592bd5e8939ede1a6ab478a6104eae

In [ ]:
with open("mergedJson.jsonl", "r") as jsonl_file:
    lines = jsonl_file.readlines()

data_list = []
for line in lines:
    data = json.loads(line)
    data_list.append(data)

# dataframe = pd.DataFrame(data_list)
dataframe = pd.DataFrame.from_dict(data_list, orient='columns')

print(dataframe)

In [ ]:
dataframe

# dataframe[0] = Description
# dataframe[1] = Data

In [ ]:
# @title Starcoder
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
# checkpoint = "bigcode/starcoder2-7b" # Took an awfully long time to download (15mb/s)
checkpoint = "bigcode/starcoder2-3b" # Crashing colab, OOM

device = "cuda" # or cuda for gpu
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(checkpoint, device_map="auto", torch_dtype=torch.bfloat16).to("cuda")

training_args = TrainingArguments(
    output_dir="./_results",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    save_steps=400,
    save_total_limit=2,
    evaluation_strategy="steps",
    eval_steps=100,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataframe,
    tokenizer=tokenizer,
)

trainer.train()

In [ ]:
# @title Phi-2/stability-code
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments

torch.set_default_device("cuda")

model = AutoModelForCausalLM.from_pretrained("stabilityai/stable-code-3b", torch_dtype="auto", trust_remote_code=True).to("cuda")
tokenizer = AutoTokenizer.from_pretrained("stabilityai/stable-code-3b", trust_remote_code=True)

training_args = TrainingArguments(
    output_dir="./_results",
    num_train_epochs=3,
    per_device_train_batch_size=1,
    save_steps=400,
    save_total_limit=2,
    evaluation_strategy="steps",
    eval_steps=100,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataframe,
    tokenizer=tokenizer,
)

trainer.train()

In [ ]:
model = None
with torch.no_grad():
    torch.cuda.empty_cache()

In [ ]:
from transformers import RobertaTokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments

tokenizer = RobertaTokenizer.from_pretrained('Salesforce/codet5-small')
model = T5ForConditionalGeneration.from_pretrained('Salesforce/codet5-small')

training_args = TrainingArguments(
    output_dir="./_results",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    save_steps=500,
    save_total_limit=2,
    evaluation_strategy="steps",
    eval_steps=100,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataframe,
    tokenizer=tokenizer,
)

trainer.train()